In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
df=pd.read_csv('data/data5_only_lag.csv')

In [4]:
len(df)

2922

In [5]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 167 columns):
 #    Column                               Non-Null Count  Dtype  
---   ------                               --------------  -----  
 0    date                                 2922 non-null   object 
 1    aqipm25                              2922 non-null   float64
 2    year                                 2922 non-null   int64  
 3    month                                2922 non-null   int64  
 4    dayofweek                            2922 non-null   int64  
 5    dayofyear                            2922 non-null   int64  
 6    weekofyear                           2922 non-null   int64  
 7    is_weekend                           2922 non-null   int64  
 8    month_sin                            2922 non-null   float64
 9    month_cos                            2922 non-null   float64
 10   doy_sin                              2922 non-null   float64
 11   doy_cos        

In [6]:
df.head(3)

,date,aqipm25,year,month,dayofweek,dayofyear,weekofyear,is_weekend,month_sin,month_cos,doy_sin,doy_cos,aqi_lag_1,aqi_lag_2,aqi_lag_3,aqi_lag_5,aqi_lag_7,aqi_lag_14,aqi_lag_21,aqi_lag_30,aqi_roll_mean_3,aqi_roll_std_3,aqi_roll_mean_7,aqi_roll_std_7,aqi_roll_mean_14,aqi_roll_std_14,aqi_roll_mean_30,aqi_roll_std_30,aqi_roll_min_7,aqi_roll_max_7,temp_lag_1,temp_lag_3,temp_lag_7,humidity_lag_1,humidity_lag_3,humidity_lag_7,dew_lag_1,dew_lag_3,dew_lag_7,windspeed_lag_1,windspeed_lag_3,windspeed_lag_7,winddir_lag_1,winddir_lag_3,winddir_lag_7,windgust_lag_1,windgust_lag_3,windgust_lag_7,precip_lag_1,precip_lag_3,precip_lag_7,precipcover_lag_1,precipcover_lag_3,precipcover_lag_7,cloudcover_lag_1,cloudcover_lag_3,cloudcover_lag_7,visibility_lag_1,visibility_lag_3,visibility_lag_7,sealevelpressure_lag_1,sealevelpressure_lag_3,sealevelpressure_lag_7,solarradiation_lag_1,solarradiation_lag_3,solarradiation_lag_7,solarenergy_lag_1,solarenergy_lag_3,solarenergy_lag_7,uvindex_lag_1,uvindex_lag_3,uvindex_lag_7,daylight_minutes_lag_1,daylight_minutes_lag_3,daylight_minutes_lag_7,preciptype_lag_1,preciptype_lag_3,preciptype_lag_7,icon_clear-day_lag_1,icon_clear-day_lag_3,icon_clear-day_lag_7,icon_partly-cloudy-day_lag_1,icon_partly-cloudy-day_lag_3,icon_partly-cloudy-day_lag_7,icon_rain_lag_1,icon_rain_lag_3,icon_rain_lag_7,tempmax_lag_1,tempmax_lag_3,tempmax_lag_7,tempmin_lag_1,tempmin_lag_3,tempmin_lag_7,moonphase_lag_1,moonphase_lag_3,moonphase_lag_7,temp_roll_mean_3,temp_roll_mean_7,temp_roll_mean_14,humidity_roll_mean_3,humidity_roll_mean_7,humidity_roll_mean_14,dew_roll_mean_3,dew_roll_mean_7,dew_roll_mean_14,windspeed_roll_mean_3,windspeed_roll_mean_7,windspeed_roll_mean_14,winddir_roll_mean_3,winddir_roll_mean_7,winddir_roll_mean_14,windgust_roll_mean_3,windgust_roll_mean_7,windgust_roll_mean_14,precip_roll_mean_3,precip_roll_mean_7,precip_roll_mean_14,precipcover_roll_mean_3,precipcover_roll_mean_7,precipcover_roll_mean_14,cloudcover_roll_mean_3,cloudcover_roll_mean_7,cloudcover_roll_mean_14,visibility_roll_mean_3,visibility_roll_mean_7,visibility_roll_mean_14,sealevelpressure_roll_mean_3,sealevelpressure_roll_mean_7,sealevelpressure_roll_mean_14,solarradiation_roll_mean_3,solarradiation_roll_mean_7,solarradiation_roll_mean_14,solarenergy_roll_mean_3,solarenergy_roll_mean_7,solarenergy_roll_mean_14,uvindex_roll_mean_3,uvindex_roll_mean_7,uvindex_roll_mean_14,daylight_minutes_roll_mean_3,daylight_minutes_roll_mean_7,daylight_minutes_roll_mean_14,preciptype_roll_mean_3,preciptype_roll_mean_7,preciptype_roll_mean_14,icon_clear-day_roll_mean_3,icon_clear-day_roll_mean_7,icon_clear-day_roll_mean_14,icon_partly-cloudy-day_roll_mean_3,icon_partly-cloudy-day_roll_mean_7,icon_partly-cloudy-day_roll_mean_14,icon_rain_roll_mean_3,icon_rain_roll_mean_7,icon_rain_roll_mean_14,tempmax_roll_mean_3,tempmax_roll_mean_7,tempmax_roll_mean_14,tempmin_roll_mean_3,tempmin_roll_mean_7,tempmin_roll_mean_14,moonphase_roll_mean_3,moonphase_roll_mean_7,moonphase_roll_mean_14,rain_days_last_3,rain_days_last_7,temp_humidity_interaction,wind_dispersion_index,stagnation_index
0,2016-01-01,107.0,2016,1,4,1,53,0,0.5,0.866025,0.017213,0.999852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,107.0,2016,1,5,2,53,1,0.5,0.866025,0.034422,0.999407,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.1,NaN,NaN,67.7,NaN,NaN,13.9,NaN,NaN,27.4,NaN,NaN,89.8,NaN,NaN,41.8,NaN,NaN,0.0

In [7]:
df['date']=pd.to_datetime(df['date'])

In [8]:
df = df.sort_values("date").reset_index(drop=True)

df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["year"] = df["date"].dt.year


# A) adding past years aqi avg on same day ,previous day and next day and also mean of all

In [9]:
import numpy as np

In [10]:
# df.iloc[2500]

In [11]:
# target_date = df.iloc[2500]["date"] + pd.Timedelta(days=-1)

# hist = df[
#     (df["month"] == target_date.month) &
#     (df["day"] == target_date.day) &
#     (df["year"] < df.iloc[2500]["year"])
# ][["aqipm25",'date']]
# hist

In [12]:
def hist_aqi_avg(df, day_offset):
    results = []

    for idx, row in df.iterrows():
        target_date = row["date"] + pd.Timedelta(days=day_offset)

        hist = df[
            (df["month"] == target_date.month) &
            (df["day"] == target_date.day) &
            (df["year"] < row["year"])
        ]["aqipm25"]

        results.append(hist.mean() if len(hist) > 0 else np.nan)

    return results


In [13]:
df["aqi_hist_prev_day_avg"] = hist_aqi_avg(df, -1)
df["aqi_hist_same_day_avg"] = hist_aqi_avg(df, 0)
df["aqi_hist_next_day_avg"] = hist_aqi_avg(df, 1)


In [14]:
df["aqi_hist_3day_window_avg"] = df[
    [
        "aqi_hist_prev_day_avg",
        "aqi_hist_same_day_avg",
        "aqi_hist_next_day_avg"
    ]
].mean(axis=1)


In [15]:
df.tail(3)

,date,aqipm25,year,month,dayofweek,dayofyear,weekofyear,is_weekend,month_sin,month_cos,doy_sin,doy_cos,aqi_lag_1,aqi_lag_2,aqi_lag_3,aqi_lag_5,aqi_lag_7,aqi_lag_14,aqi_lag_21,aqi_lag_30,aqi_roll_mean_3,aqi_roll_std_3,aqi_roll_mean_7,aqi_roll_std_7,aqi_roll_mean_14,aqi_roll_std_14,aqi_roll_mean_30,aqi_roll_std_30,aqi_roll_min_7,aqi_roll_max_7,temp_lag_1,temp_lag_3,temp_lag_7,humidity_lag_1,humidity_lag_3,humidity_lag_7,dew_lag_1,dew_lag_3,dew_lag_7,windspeed_lag_1,windspeed_lag_3,windspeed_lag_7,winddir_lag_1,winddir_lag_3,winddir_lag_7,windgust_lag_1,windgust_lag_3,windgust_lag_7,precip_lag_1,precip_lag_3,precip_lag_7,precipcover_lag_1,precipcover_lag_3,precipcover_lag_7,cloudcover_lag_1,cloudcover_lag_3,cloudcover_lag_7,visibility_lag_1,visibility_lag_3,visibility_lag_7,sealevelpressure_lag_1,sealevelpressure_lag_3,sealevelpressure_lag_7,solarradiation_lag_1,solarradiation_lag_3,solarradiation_lag_7,solarenergy_lag_1,solarenergy_lag_3,solarenergy_lag_7,uvindex_lag_1,uvindex_lag_3,uvindex_lag_7,daylight_minutes_lag_1,daylight_minutes_lag_3,daylight_minutes_lag_7,preciptype_lag_1,preciptype_lag_3,preciptype_lag_7,icon_clear-day_lag_1,icon_clear-day_lag_3,icon_clear-day_lag_7,icon_partly-cloudy-day_lag_1,icon_partly-cloudy-day_lag_3,icon_partly-cloudy-day_lag_7,icon_rain_lag_1,icon_rain_lag_3,icon_rain_lag_7,tempmax_lag_1,tempmax_lag_3,tempmax_lag_7,tempmin_lag_1,tempmin_lag_3,tempmin_lag_7,moonphase_lag_1,moonphase_lag_3,moonphase_lag_7,temp_roll_mean_3,temp_roll_mean_7,temp_roll_mean_14,humidity_roll_mean_3,humidity_roll_mean_7,humidity_roll_mean_14,dew_roll_mean_3,dew_roll_mean_7,dew_roll_mean_14,windspeed_roll_mean_3,windspeed_roll_mean_7,windspeed_roll_mean_14,winddir_roll_mean_3,winddir_roll_mean_7,winddir_roll_mean_14,windgust_roll_mean_3,windgust_roll_mean_7,windgust_roll_mean_14,precip_roll_mean_3,precip_roll_mean_7,precip_roll_mean_14,precipcover_roll_mean_3,precipcover_roll_mean_7,precipcover_roll_mean_14,cloudcover_roll_mean_3,cloudcover_roll_mean_7,cloudcover_roll_mean_14,visibility_roll_mean_3,visibility_roll_mean_7,visibility_roll_mean_14,sealevelpressure_roll_mean_3,sealevelpressure_roll_mean_7,sealevelpressure_roll_mean_14,solarradiation_roll_mean_3,solarradiation_roll_mean_7,solarradiation_roll_mean_14,solarenergy_roll_mean_3,solarenergy_roll_mean_7,solarenergy_roll_mean_14,uvindex_roll_mean_3,uvindex_roll_mean_7,uvindex_roll_mean_14,daylight_minutes_roll_mean_3,daylight_minutes_roll_mean_7,daylight_minutes_roll_mean_14,preciptype_roll_mean_3,preciptype_roll_mean_7,preciptype_roll_mean_14,icon_clear-day_roll_mean_3,icon_clear-day_roll_mean_7,icon_clear-day_roll_mean_14,icon_partly-cloudy-day_roll_mean_3,icon_partly-cloudy-day_roll_mean_7,icon_partly-cloudy-day_roll_mean_14,icon_rain_roll_mean_3,icon_rain_roll_mean_7,icon_rain_roll_mean_14,tempmax_roll_mean_3,tempmax_roll_mean_7,tempmax_roll_mean_14,tempmin_roll_mean_3,tempmin_roll_mean_7,tempmin_roll_mean_14,moonphase_roll_mean_3,moonphase_roll_mean_7,moonphase_roll_mean_14,rain_days_last_3,rain_days_last_7,temp_humidity_interaction,wind_dispersion_index,stagnation_index,day,aqi_hist_prev_day_avg,aqi_hist_same_day_avg,aqi_hist_next_day_avg,aqi_hist_3day_window_avg
2919,2025-12-29,153.000000,2025,12,0,363,1,0,-2.449294e-16,1.0,-3.442161e-02,0.999407,153.0,129.0,142.0,147.0,148.0,145.0,155.0,152.0,141.333333,12.013881,144.285714,7.565586,144.714286,8.222189,143.233333,9.679995,129.0,153.0,20.1,20.5,19.3,74.0,74.2,73.9,14.9,15.3,14.2,16.5,18.3,18.5,83.7,87.3,77.0,23.0,28.1,31.3,0.0,0.0,0.0,0.0,0.0,0.0,38.5,43.8,41.7,3.9,4.1,4.1,1017.6,1016.9,1016.9,243.1,204.7,236.3,21.0,17.5,20.4,9.0,8.0,9.0,682.233333,682.033333,681.850000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,25.9,26.0,24.9,15.9,16.9,14.9,0.25,0.22,0.08,20.233333,19.928571,19.900000,71.900000,70.828571,70.814286,14.433333,13.785714,13.685714,17.133333,16.771429,17.950000,82.233333,80.485714,79.642857,25.900000,26.571429,29.321429,0.0,0.0,0.0,0.0,0.0,0.0,39.466667,39.714286,39.185714,3.866667,4.042857,

In [16]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 172 columns):
 #    Column                               Non-Null Count  Dtype         
---   ------                               --------------  -----         
 0    date                                 2922 non-null   datetime64[ns]
 1    aqipm25                              2922 non-null   float64       
 2    year                                 2922 non-null   int32         
 3    month                                2922 non-null   int32         
 4    dayofweek                            2922 non-null   int64         
 5    dayofyear                            2922 non-null   int64         
 6    weekofyear                           2922 non-null   int64         
 7    is_weekend                           2922 non-null   int64         
 8    month_sin                            2922 non-null   float64       
 9    month_cos                            2922 non-null   float64       
 10 

# A) adding is holiday feature

In [17]:
import holidays

india_holidays = holidays.India()

df["is_holiday"] = df["date"].apply(
    lambda x: 1 if x in india_holidays else 0
)


In [18]:
df['is_holiday'].value_counts()

is_holiday
0    2796
1     126
Name: count, dtype: int64

In [19]:
df[df['is_holiday']==1].head(3)

,date,aqipm25,year,month,dayofweek,dayofyear,weekofyear,is_weekend,month_sin,month_cos,doy_sin,doy_cos,aqi_lag_1,aqi_lag_2,aqi_lag_3,aqi_lag_5,aqi_lag_7,aqi_lag_14,aqi_lag_21,aqi_lag_30,aqi_roll_mean_3,aqi_roll_std_3,aqi_roll_mean_7,aqi_roll_std_7,aqi_roll_mean_14,aqi_roll_std_14,aqi_roll_mean_30,aqi_roll_std_30,aqi_roll_min_7,aqi_roll_max_7,temp_lag_1,temp_lag_3,temp_lag_7,humidity_lag_1,humidity_lag_3,humidity_lag_7,dew_lag_1,dew_lag_3,dew_lag_7,windspeed_lag_1,windspeed_lag_3,windspeed_lag_7,winddir_lag_1,winddir_lag_3,winddir_lag_7,windgust_lag_1,windgust_lag_3,windgust_lag_7,precip_lag_1,precip_lag_3,precip_lag_7,precipcover_lag_1,precipcover_lag_3,precipcover_lag_7,cloudcover_lag_1,cloudcover_lag_3,cloudcover_lag_7,visibility_lag_1,visibility_lag_3,visibility_lag_7,sealevelpressure_lag_1,sealevelpressure_lag_3,sealevelpressure_lag_7,solarradiation_lag_1,solarradiation_lag_3,solarradiation_lag_7,solarenergy_lag_1,solarenergy_lag_3,solarenergy_lag_7,uvindex_lag_1,uvindex_lag_3,uvindex_lag_7,daylight_minutes_lag_1,daylight_minutes_lag_3,daylight_minutes_lag_7,preciptype_lag_1,preciptype_lag_3,preciptype_lag_7,icon_clear-day_lag_1,icon_clear-day_lag_3,icon_clear-day_lag_7,icon_partly-cloudy-day_lag_1,icon_partly-cloudy-day_lag_3,icon_partly-cloudy-day_lag_7,icon_rain_lag_1,icon_rain_lag_3,icon_rain_lag_7,tempmax_lag_1,tempmax_lag_3,tempmax_lag_7,tempmin_lag_1,tempmin_lag_3,tempmin_lag_7,moonphase_lag_1,moonphase_lag_3,moonphase_lag_7,temp_roll_mean_3,temp_roll_mean_7,temp_roll_mean_14,humidity_roll_mean_3,humidity_roll_mean_7,humidity_roll_mean_14,dew_roll_mean_3,dew_roll_mean_7,dew_roll_mean_14,windspeed_roll_mean_3,windspeed_roll_mean_7,windspeed_roll_mean_14,winddir_roll_mean_3,winddir_roll_mean_7,winddir_roll_mean_14,windgust_roll_mean_3,windgust_roll_mean_7,windgust_roll_mean_14,precip_roll_mean_3,precip_roll_mean_7,precip_roll_mean_14,precipcover_roll_mean_3,precipcover_roll_mean_7,precipcover_roll_mean_14,cloudcover_roll_mean_3,cloudcover_roll_mean_7,cloudcover_roll_mean_14,visibility_roll_mean_3,visibility_roll_mean_7,visibility_roll_mean_14,sealevelpressure_roll_mean_3,sealevelpressure_roll_mean_7,sealevelpressure_roll_mean_14,solarradiation_roll_mean_3,solarradiation_roll_mean_7,solarradiation_roll_mean_14,solarenergy_roll_mean_3,solarenergy_roll_mean_7,solarenergy_roll_mean_14,uvindex_roll_mean_3,uvindex_roll_mean_7,uvindex_roll_mean_14,daylight_minutes_roll_mean_3,daylight_minutes_roll_mean_7,daylight_minutes_roll_mean_14,preciptype_roll_mean_3,preciptype_roll_mean_7,preciptype_roll_mean_14,icon_clear-day_roll_mean_3,icon_clear-day_roll_mean_7,icon_clear-day_roll_mean_14,icon_partly-cloudy-day_roll_mean_3,icon_partly-cloudy-day_roll_mean_7,icon_partly-cloudy-day_roll_mean_14,icon_rain_roll_mean_3,icon_rain_roll_mean_7,icon_rain_roll_mean_14,tempmax_roll_mean_3,tempmax_roll_mean_7,tempmax_roll_mean_14,tempmin_roll_mean_3,tempmin_roll_mean_7,tempmin_roll_mean_14,moonphase_roll_mean_3,moonphase_roll_mean_7,moonphase_roll_mean_14,rain_days_last_3,rain_days_last_7,temp_humidity_interaction,wind_dispersion_index,stagnation_index,day,aqi_hist_prev_day_avg,aqi_hist_same_day_avg,aqi_hist_next_day_avg,aqi_hist_3day_window_avg,is_holiday
25,2016-01-26,226.000000,2016,1,1,26,4,0,0.5,8.660254e-01,0.432776,0.901502,147.0,127.0,159.0,151.0,161.0,150.000000,134.0,NaN,144.333333,16.165808,154.428571,14.443651,156.785714,11.095242,NaN,NaN,127.0,168.0,23.7,22.1,21.8,74.8,77.6,77.6,18.5,17.6,17.5,11.0,14.3,11.2,110.0,90.8,126.0,25.9,33.8,25.9,0.0,0.2,0.4,0.0,8.33,4.17,48.6,48.3,76.0,5.4,5.0,5.3,1017.8,1018.6,1014.7,213.2,139.0,128.7,18.5,12.1,11.0,9.0,5.0,6.0,690.883333,689.916667,688.116667,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,31.5,26.1,25.0,19.1,18.1,18.1,0.54,0.47,0.33,22.800000,22.400000,21.771429,76.733333,79.271429,72.128571,18.066667,18.285714,15.785714,14.5,13.800000,14.107143,98.666667,103.414286,101.642857,29.366667,26.671429,27.885714,0.1,1.107857,0.553929,4.166667,9.524286,4.762143,48.933333,60.571429,43.45000

In [ ]:
df["is_pre_holiday"] = (
    df["date"] + pd.Timedelta(days=1)
).dt.date.isin(india_holidays).astype(int)

df["is_post_holiday"] = (
    df["date"] - pd.Timedelta(days=1)
).dt.date.isin(india_holidays).astype(int)


# saving files

In [21]:
df.to_csv('data/data7_only_lag_addon_features.csv',index=False)

In [22]:
len(df)

2922

In [23]:
df1=df.dropna()
len(df1)

2526

In [24]:
2922-366-30

2526

In [25]:
df1.to_csv('data/data8_only_lag_addon_features_nona.csv',index=False)